In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics

# 设置GPU使用方式
# 获取GPU列表
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # 设置GPU为增长式占用
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True) 
  except RuntimeError as e:
    # 打印异常
    print(e)

(xs, ys),_ = datasets.mnist.load_data()
print('datasets:', xs.shape, ys.shape, xs.min(), xs.max())

batch_size = 32

xs = tf.convert_to_tensor(xs, dtype=tf.float32) / 255.
db = tf.data.Dataset.from_tensor_slices((xs,ys))
db = db.batch(batch_size).repeat(30)

model = Sequential([layers.Dense(256, activation='relu'), 
                     layers.Dense(128, activation='relu'),
                     layers.Dense(10)])
model.build(input_shape=(4, 28*28))
model.summary()

11501568/11490434 [==============================] - 0s 0us/step
datasets: (60000, 28, 28) (60000,) 0 255
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (4, 256)                  200960    
_________________________________________________________________
dense_1 (Dense)              (4, 128)                  32896     
_________________________________________________________________
dense_2 (Dense)              (4, 10)                   1290      
Total params: 235,146
Trainable params: 235,146
Non-trainable params: 0
_________________________________________________________________


In [3]:
optimizer = optimizers.SGD(lr=0.01)
acc_meter = metrics.Accuracy()

for step, (x,y) in enumerate(db):

    with tf.GradientTape() as tape:
        # 打平操作，[b, 28, 28] => [b, 784]
        x = tf.reshape(x, (-1, 28*28))
        # Step1. 得到模型输出output [b, 784] => [b, 10]
        out = model(x)
        # [b] => [b, 10]
        y_onehot = tf.one_hot(y, depth=10)
        # 计算差的平方和，[b, 10]
        loss = tf.square(out-y_onehot)
        # 计算每个样本的平均误差，[b]
        loss = tf.reduce_sum(loss) / x.shape[0]

    acc_meter.update_state(tf.argmax(out, axis=1), y)

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))


    if step % 200==0:

        print(step, 'loss:', float(loss), 'acc:', acc_meter.result().numpy())
        acc_meter.reset_states()

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


0 loss: 1.7181165218353271 acc: 0.03125
200 loss: 0.45764419436454773 acc: 0.6690625
400 loss: 0.4112520217895508 acc: 0.83421874
600 loss: 0.3355519473552704 acc: 0.8575
800 loss: 0.293040931224823 acc: 0.8854687
1000 loss: 0.3779064416885376 acc: 0.8871875
1200 loss: 0.31504568457603455 acc: 0.9015625
1400 loss: 0.2666063904762268 acc: 0.91234374
1600 loss: 0.2240939438343048 acc: 0.904375
1800 loss: 0.22073759138584137 acc: 0.92359376
2000 loss: 0.22105976939201355 acc: 0.93859375
2200 loss: 0.17192360758781433 acc: 0.9248437
2400 loss: 0.2514965832233429 acc: 0.9209375
2600 loss: 0.2396107316017151 acc: 0.9284375
2800 loss: 0.18501240015029907 acc: 0.9298437
3000 loss: 0.26290035247802734 acc: 0.92640626
3200 loss: 0.20434601604938507 acc: 0.9321875
3400 loss: 0.16104988753795624 acc: 0.931875
3600 loss: 0.11298942565917969 acc: 0.93296874
3800 loss: 0.2167688012123108 acc: 0.951875
4000 loss: 0.2099430114030838 acc: 0.9490625
4200 loss: 0.17210018634796143 acc: 0.93296874
4400 los